In [1]:
# - Prepare the input data
# - Define network architecture
# - Define loss and optimizer
# - Train
# - Test vs. training set
# - Test vs. holdout set

import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn import preprocessing

/Users/freddiekarlbom/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#flags = tf.app.flags
#FLAGS = flags.FLAGS

In [3]:
# - Prepare the input data
df = pd.read_csv('./data/wine_data.csv', header=None)
columns = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids',
           'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
           'Proline']
df.columns = columns

In [4]:
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [5]:
sess = tf.InteractiveSession()

X = df[df.columns[1:13]].values
y = df['Class']
Y = tf.one_hot(indices=y, depth=3, on_value=1., off_value=0.).eval()

In [6]:
X, Y = shuffle(X, Y)

In [7]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

In [8]:
# - Define network architecture
x = tf.placeholder(tf.float32, [None, 12])
W = tf.Variable(tf.zeros([12, 3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [9]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 3])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

learning_rate = 0.1
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [27]:
# - Train
tf.initialize_all_variables().run()

# @todo: Replace with sklearn train_test_split
X, Y = shuffle(X, Y, random_state=1)
X_train = X[0:140,:]
Y_train = Y[0:140,:]
X_test = X[140:178,:]
Y_test = Y[140:178,:]

for i in range(100):
    # This would make sense for minibatches...
    #X_train, Y_train = shuffle(X_train, Y_train, random_state=0)    

    train_step.run({x: X_train, y_: Y_train})
    cost = sess.run(cross_entropy, feed_dict={x: X_train, y_: Y_train})

print("Cost: {}".format(cost))

Cost: 0.08923417329788208


In [28]:
# - Test vs. training set
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: X_train, y_: Y_train}))

0.9071429


In [31]:
# - Test vs. holdout set
#predictions = sess.run(y, feed_dict={x: X_test})
#tf.argmax(predictions, 1)

<tf.Tensor 'ArgMax_10:0' shape=(38,) dtype=int64>

### Takeaways
- Need to dig deeper into the loss function
- Need to complete how to evaluate vs. holdout set as well...